![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FRetrieval&file=Retrieval+-+BigQuery+Vector+Indexing+And+Search.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20BigQuery%20Vector%20Indexing%20And%20Search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FRetrieval%2FRetrieval%2520-%2520BigQuery%2520Vector%2520Indexing%2520And%2520Search.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20BigQuery%20Vector%20Indexing%20And%20Search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Retrieval/Retrieval%20-%20BigQuery%20Vector%20Indexing%20And%20Search.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Retrieval - BigQuery Vector Indexing And Search

In prior workflows, a series of documents was [processed into chunks](../Chunking/readme.md), and for each chunk, [embeddings](../Embeddings/readme.md) were created:

- Process: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb)
- Embed: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

Retrieving chunks for a query involves calculating the embedding for the query and then using similarity metrics to find relevant chunks. A thorough review of similarity matching can be found in [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb) - use dot product! As development moves from experiment to application, the process of storing and computing similarity is migrated to a [retrieval](./readme.md) system. This workflow is part of a [series of workflows exploring many retrieval systems](./readme.md).  

A detailed [comparison of many retrieval systems](./readme.md#comparison-of-vector-database-solutions) can be found in the readme as well.

---

**BigQuery For Storage, Indexing, And Search**

This workflow builds a retrieval system using Google [BigQuery](https://cloud.google.com/bigquery). BigQuery is a fully managed data warehouse where SQL queries run without the need to plan for storage or compute requirements.  Built into this solution is the  `VECTOR_SEARCH` function that can perform brute-force searches for neighboring embeddings and utilize an index for efficient search. BigQuery offers two built-in methods for [creating vector indexes](https://cloud.google.com/bigquery/docs/vector-index): the [Inverted File (IVF) index](https://cloud.google.com/bigquery/docs/vector-index#ivf-index) and the [TreeAH index](https://cloud.google.com/bigquery/docs/vector-index#tree-ah-index).

---

**Use Case Data**

Buying a home usually involves borrowing money from a lending institution, typically through a mortgage secured by the home's value. But how do these institutions manage the risks associated with such large loans, and how are lending standards established?

In the United States, two government-sponsored enterprises (GSEs) play a vital role in the housing market:

- Federal National Mortgage Association ([Fannie Mae](https://www.fanniemae.com/))
- Federal Home Loan Mortgage Corporation ([Freddie Mac](https://www.freddiemac.com/))

These GSEs purchase mortgages from lenders, enabling those lenders to offer more loans. This process also allows Fannie Mae and Freddie Mac to set standards for mortgages, ensuring they are responsible and borrowers are more likely to repay them. This system makes homeownership more affordable and stabilizes the housing market by maintaining a steady flow of liquidity for lenders and keeping interest rates controlled.

However, navigating the complexities of these GSEs and their extensive servicing guides can be challenging.

**Approaches**

[This series](../readme.md) covers many generative AI workflows. These documents are used directly as long context for Gemini in the workflow [Long Context Retrieval With The Vertex AI Gemini API](../Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb). The workflow below uses a [retrieval](./readme.md) approach with the already generated chunks and embeddings.

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('google.cloud.bigquery', 'google-cloud-bigquery')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'retrieval-bigquery'

# make this the BigQuery Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = EXPERIMENT
BQ_REGION = REGION[0:2]

Packages

In [8]:
import os, json, time, glob

import numpy as np

# Vertex AI
from google.cloud import aiplatform
import vertexai.language_models # for embeddings API
import vertexai.generative_models # for Gemini Models

# BigQuery
from google.cloud import bigquery

In [9]:
aiplatform.__version__

'1.71.0'

Clients

In [10]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

---
## Text & Embeddings For Examples

This repository contains a [section for document processing (chunking)](../Chunking/readme.md) that includes an example of processing mulitple large pdfs (over 1000 pages) into chunks: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb).  The chunks of text from that workflow are stored with this repository and loaded by another companion workflow that augments the chunks with text embeddings: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb).

The following code will load the version of the chunks that includes text embeddings and prepare it for a local example of retrival augmented generation.

### Get The Documents

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [11]:
local_dir = '../Embeddings/files/embeddings-api'

In [12]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Embeddings/files/embeddings-api', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../Embeddings/files/embeddings-api`


### Load The Chunks

In [13]:
jsonl_files = glob.glob(f"{local_dir}/large-files*.jsonl")
jsonl_files.sort()
jsonl_files

['../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0000.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0001.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0002.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0003.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0004.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0005.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0006.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0007.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0008.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0009.jsonl']

In [14]:
chunks = []
for file in jsonl_files:
    with open(file, 'r') as f:
        chunks.extend([json.loads(line) for line in f])
len(chunks)

9040

### Review A Chunk

In [15]:
chunks[0].keys()

dict_keys(['instance', 'predictions', 'status'])

In [16]:
chunks[0]['instance']['chunk_id']

'fannie_part_0_c17'

In [17]:
print(chunks[0]['instance']['content'])

# Selling Guide Fannie Mae Single Family

## Fannie Mae Copyright Notice

### Fannie Mae Copyright Notice

|-|
| Section B3-4.2, Verification of Depository Assets 402 |
| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |
| B3-4.2-02, Depository Accounts (12/14/2022) 405 |
| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |
| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |
| B3-4.2-05, Foreign Assets (05/04/2022) 411 |
| Section B3-4.3, Verification of Non-Depository Assets 412 |
| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |
| B3-4.3-02, Trust Accounts (04/01/2009) 413 |
| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |
| B3-4.3-04, Personal Gifts (09/06/2023) 415 |
| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |
| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |
| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423 |
| B3-4.3-08, Employer Assistance (09/29/2015) 423 |
| B3-4.3-09,

In [18]:
chunks[0]['predictions'][0]['embeddings']['values'][0:10]

[0.031277116388082504,
 0.03056905046105385,
 0.010865348391234875,
 0.0623614676296711,
 0.03228681534528732,
 0.05066155269742012,
 0.046544693410396576,
 0.05509665608406067,
 -0.014074751175940037,
 0.008380400016903877]

### Prepare Chunk Structure

Make a list of dictionaries with information for each chunk:

In [19]:
content_chunks = [
    dict(
        gse = chunk['instance']['gse'],
        chunk_id = chunk['instance']['chunk_id'],
        content = chunk['instance']['content'],
        embedding = chunk['predictions'][0]['embeddings']['values']
    ) for chunk in chunks
]

### Query Embedding

Create a query, or prompt, and get the embedding for it:

Connect to models for text embeddings. Learn more about the model API:
- [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

In [20]:
question = "Does a lender have to perform servicing functions directly?"

In [21]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')

In [22]:
question_embedding = embedder.get_embeddings([question])[0].values
question_embedding[0:10]

[-0.0005117303808219731,
 0.009651427157223225,
 0.01768726110458374,
 0.014538003131747246,
 -0.01829824410378933,
 0.027877431362867355,
 -0.021124685183167458,
 0.008830446749925613,
 -0.02669006586074829,
 0.06414774805307388]

---
## Load To BigQuery

In this case the information to load to BigQuery is local.  It could be in GCS or other BigQuery sources.  You can also get embeddings for information within BigQuery using the [ML.GENERATE_EMBEDDING function](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-embedding) and even use the exact same model as was used for the data imported above.

### Create/Recall Dataset

In [39]:
dataset = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
dataset.location = BQ_REGION
bq_dataset = bq.create_dataset(dataset, exists_ok = True)

### Load JSON TO BigQuery Table

In [40]:
bq_table = bq_dataset.table(BQ_TABLE)

In [41]:
job_config = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE,
    autodetect = True
)

In [42]:
load_job = bq.load_table_from_json(
    json_rows = content_chunks,
    destination = bq_table,
    job_config = job_config
)
load_job.result()

LoadJob<project=statmike-mlops-349915, location=US, id=5827dba5-288e-4f93-bb2e-25a4783b8d30>

In [43]:
bq.query(f"SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5").to_dataframe()

,chunk_id,embedding,content,gse
0,fannie_part_0_c17,"[0.031277116388082504, 0.03056905046105385, 0....",# Selling Guide Fannie Mae Single Family\n\n##...,fannie
1,fannie_part_0_c418,"[0.0002988415362779051, -0.002309585688635707,...",# Additional Financial Requirements\n\nLender ...,fannie
2,fannie_part_0_c725,"[-0.0030626351945102215, -0.017544567584991455...","# B3-1-01, Comprehensive Risk Assessment (12/1...",fannie
3,fannie_part_0_c882,"[0.012272126041352749, -0.050955940037965775, ...",# Calculating Monthly Qualifying Rental Income...,fannie
4,fannie_part_0_c315,"[0.008982143364846706, -0.0069817849434912205,...",# Compliance with Fannie Mae Data Breach Incid...,fannie


---
## Retrieval With BigQuery

BigQuery has a built in [VECTOR_SEARCH function](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search) that by itself will do brute force matching to find the `top_k` neighbors with the selected distance metric using `distance_type` ('EUCLIDEAN', 'COSINE', or 'DOT_PRODUCT').  Remember to use 'DOT_PRODUCT' and read why in this companion workflow: [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb).

[BigQuery also has vector indexing](https://cloud.google.com/bigquery/docs/vector-search-intro) options for approximate nearest neighbor search with two options for index type:
- [Inverted File (IVF) Index](https://cloud.google.com/bigquery/docs/vector-index#ivf-index)
    - Automatically creates an inverted list from assigning embeddings to k clusters using k-means clustering and uses the clusters as partions
    - Reduces the search space to only partions (clusters) near the query embedding (configurable)
- [TreeAH Index](https://cloud.google.com/bigquery/docs/vector-index#tree-ah-index)
    - Uses the Google [ScaNN algorithm](https://github.com/google-research/google-research/blob/master/scann/docs/algorithms.md)
    - Train a clustering model and bases the number of clusters on the value of `leaf_node_embedding_count`
    - Creates a candidate list at query time using asymmetric hashing - super fast!
    
The following sections explore vector search in BigQuery with and without the use of vector indexes.

### Get Matches: Vector Search With No Vector Index - Brute Force

The [VECTOR_SEARCH function](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search) function can be directly used to find matches without a [vector index](https://cloud.google.com/bigquery/docs/vector-index).  While vector indexes offer efficient ways of finding matches based on embeddings, the brute force approach of directly using `VECTOR_SEARCH` compares a query embedding to all embeddings in the table.

The options for `VECTOR_SEARCH` are:
- `base_table` is the table to search
- `column_to_search` is the column of embeddings vector to match
- `top_k` is the number of matches to return
- `distance_type` is the distance measure to use from `EUCLIDEAN`, `DOT_PRODUCT`, and `COSINE`.
    - Use `DOT_PRODUCT` here for these normalized embeddings and read more about why in [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb)

#### Matches For A Given `chunk_id`

In [44]:
example_chunk = 'fannie_part_0_c17'
query = f'''
SELECT
    query.chunk_id AS chunk_id,
    base.chunk_id AS match,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE chunk_id = '{example_chunk}'),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT'
)
ORDER BY distance
'''
bq.query(query).to_dataframe()

,chunk_id,match,distance
0,fannie_part_0_c17,fannie_part_0_c17,-0.999799
1,fannie_part_0_c17,fannie_part_0_c15,-0.897096
2,fannie_part_0_c17,fannie_part_0_c20,-0.889913
3,fannie_part_0_c17,fannie_part_0_c11,-0.883223
4,fannie_part_0_c17,fannie_part_0_c22,-0.877935


#### Matches For An Input Embedding

In [45]:
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT'
)
ORDER BY distance
'''
bq.query(query).to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,freddie_part_4_c509,freddie,-0.680526
2,Does a lender have to perform servicing functi...,freddie_part_4_c510,freddie,-0.675330
3,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
4,Does a lender have to perform servicing functi...,fannie_part_0_c326,fannie,-0.668350


#### Pre-Filter Matches For An Input Embedding

In [46]:
example_gse = 'fannie'
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE gse = '{example_gse}'),
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT'
)
ORDER BY distance
'''
bq.query(query).to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
2,Does a lender have to perform servicing functi...,fannie_part_0_c326,fannie,-0.668350
3,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,-0.661434
4,Does a lender have to perform servicing functi...,fannie_part_0_c240,fannie,-0.660858


### Create A Vector Index

To efficiently do a vector search it can be helpful to [create a vector index](https://cloud.google.com/bigquery/docs/vector-index) for the embeddings column.  
 - This is not required though as a [brute force search](https://cloud.google.com/bigquery/docs/vector-search#use_the_vector_search_function_with_brute_force) is possible as shown in the previous section.  An Index can also be ignored and force brute force search as will be shown later in this workflow.

There are two types of vector indexes that can be chosen in BigQuery:
- IVF Index: `index_type = "IVF"`
    - use k-means to cluser the embeddings vectors and then uses the clusters as partions
    - k can be specifed with the NUM_LISTS option as any INT64 <= 5000
    - updates are handled automatically
- TreeAH Index `index_type = "TREE_AH"`
    - uses the [ScaNN algorithm](https://github.com/google-research/google-research/blob/master/scann/docs/algorithms.md) developed by Google
    - the number of partitions/clusters is trained based on the `leaf_node_embedding_count` option
        - each leaf node will contain up to the value of the parameter which defaults to 1000 and can be any INT64 >= 500
    
Options for both index types:
- Set the default `DISTANCE_TYPE` as `EUCLIDEAN`, `DOT_PRODUCT` or `COSINE`.
    - Use `DOT_PRODUCT` here for these normalized embeddings and read more about why in [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb)
    - This option can be preempted with an alternative choice when using the VECTOR_SEARCH function
- `NORMALIZATION_TYPE` can be chosen to normalize the vectors for you.
    - In this case the embedding are already normalized
    - The default is `NONE` but can be set to `L2`
- `STORING` Clause can be used to set additional columns as part of the index which can be helpful for pre-filtering searchs to a subset.  This is used below to enable subsetting the search to the source of the information based on the columns `gse` which has values 'freddie' and 'fannie'

Read about the [CREATE VECTOR INDEX](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-definition-language#create_vector_index_statement) DDL statement.

The workflow below uses the `IVF` index type.  See an example of the workflow below using the `TREE_AH` index type in the workflow **[BQML Autoencoder As Table Embedding](../Embeddings/BQML%20Autoencoder%20As%20Table%20Embedding.ipynb)**


In [47]:
query = f'''
CREATE VECTOR INDEX IF NOT EXISTS row_index ON `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`(embedding)
STORING(gse)
OPTIONS(
    index_type = 'IVF',
    ivf_options = '{{"num_lists":1000}}',
    distance_type = 'DOT_PRODUCT'
)
'''
job = bq.query(query)
job.result()

### Check The Vector Index Status

[Get information about vector indexes](https://cloud.google.com/bigquery/docs/vector-index#get_information_about_vector_indexes)

In [48]:
query = f'''
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.VECTOR_INDEXES`
WHERE index_status = 'ACTIVE'
    AND table_name = '{BQ_TABLE}'
'''
bq.query(query).to_dataframe()

,index_catalog,index_schema,table_name,index_name,index_status,creation_time,last_modification_time,last_refresh_time,disable_time,disable_reason,ddl,coverage_percentage,unindexed_row_count,total_logical_bytes,total_storage_bytes
0,statmike-mlops-349915,applied_genai,retrieval-bigquery,row_index,ACTIVE,2024-11-10 13:41:37.105000+00:00,2024-11-10 13:41:37.105000+00:00,NaT,NaT,None,CREATE VECTOR INDEX `row_index` ON `statmike-m...,0,9040,0,0


It can take a few minutes for the index to be built.  Notice the output above show the `index_status = ACTIVE` but the `coverage_percentage = 0`.  The next cell will for a wait of 10 minutes then rerun the check for status:

In [49]:
time.sleep(600) # sleep for 10 minutes

In [50]:
query = f'''
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.VECTOR_INDEXES`
WHERE index_status = 'ACTIVE'
    AND table_name = '{BQ_TABLE}'
'''
bq.query(query).to_dataframe()

,index_catalog,index_schema,table_name,index_name,index_status,creation_time,last_modification_time,last_refresh_time,disable_time,disable_reason,ddl,coverage_percentage,unindexed_row_count,total_logical_bytes,total_storage_bytes
0,statmike-mlops-349915,applied_genai,retrieval-bigquery,row_index,ACTIVE,2024-11-10 13:41:37.105000+00:00,2024-11-10 13:41:37.105000+00:00,2024-11-10 13:49:03.335000+00:00,NaT,None,CREATE VECTOR INDEX `row_index` ON `statmike-m...,100,0,62079448,34282075


### Get Matches: Vector Search With Vector Index

Directly query the table with the index as the base table:

#### Matches For A Given `chunk_id`

In [51]:
example_chunk = 'fannie_part_0_c17'
query = f'''
SELECT
    query.chunk_id AS chunk_id,
    base.chunk_id AS match,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE chunk_id = '{example_chunk}'),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,chunk_id,match,distance
0,fannie_part_0_c17,fannie_part_0_c17,-0.999799
1,fannie_part_0_c17,fannie_part_0_c15,-0.897096
2,fannie_part_0_c17,fannie_part_0_c20,-0.889913
3,fannie_part_0_c17,fannie_part_0_c11,-0.883223
4,fannie_part_0_c17,fannie_part_0_c22,-0.877935


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [52]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Matches For An Input Embedding

In [53]:
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
2,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,-0.661434
3,Does a lender have to perform servicing functi...,fannie_part_0_c335,fannie,-0.652192
4,Does a lender have to perform servicing functi...,fannie_part_0_c337,fannie,-0.637705


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [54]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Pre-Filter Matches For An Input Embedding

In [55]:
example_gse = 'fannie'
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE gse = '{example_gse}'),
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
2,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,-0.661434
3,Does a lender have to perform servicing functi...,fannie_part_0_c335,fannie,-0.652192
4,Does a lender have to perform servicing functi...,fannie_part_0_c337,fannie,-0.637705


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [56]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

### Get Matches: Vector Search With Vector Index - Modify Distance Metric

When setting up the index we specified `distance_type = 'DOT_PRODUCT'`.  We can modify the vector search to use a different distance measure by setting the option for `distance_type`. See [VECTOR_SEARCH function details](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search).  Does this modification still use the index?  Yes!

#### Matches For A Given `chunk_id`

In [71]:
example_chunk = 'fannie_part_0_c17'
query = f'''
SELECT
    query.chunk_id AS chunk_id,
    base.chunk_id AS match,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE chunk_id = '{example_chunk}'),
    'embedding',
    top_k => 5,
    distance_type => 'COSINE'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,chunk_id,match,distance
0,fannie_part_0_c17,fannie_part_0_c17,-2.220446e-16
1,fannie_part_0_c17,fannie_part_0_c15,1.027482e-01
2,fannie_part_0_c17,fannie_part_0_c20,1.099197e-01
3,fannie_part_0_c17,fannie_part_0_c11,1.166230e-01
4,fannie_part_0_c17,fannie_part_0_c22,1.218969e-01


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [72]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Matches For An Input Embedding

In [73]:
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'COSINE'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,0.289998
1,Does a lender have to perform servicing functi...,freddie_part_4_c509,freddie,0.319444
2,Does a lender have to perform servicing functi...,freddie_part_4_c510,freddie,0.324653
3,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,0.327604
4,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,0.338554


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [74]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Pre-Filter Matches For An Input Embedding

In [75]:
example_gse = 'fannie'
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE gse = '{example_gse}'),
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'COSINE'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,0.289998
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,0.327604
2,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,0.338554
3,Does a lender have to perform servicing functi...,fannie_part_0_c335,fannie,0.347782
4,Does a lender have to perform servicing functi...,fannie_part_0_c337,fannie,0.362269


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [76]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

### Get Matches: Vector Search With Vector Index - Modify Search Size

When setting up the index we specified `num_lists = 1000` which led to a number of list being created.  We can guide the vector search to use a larger/smaller portion of these list by seting the option for `fraction_list_to_search`. See [VECTOR_SEARCH function details](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search).

#### Matches For A Given `chunk_id`

In [77]:
example_chunk = 'fannie_part_0_c17'
query = f'''
SELECT
    query.chunk_id AS chunk_id,
    base.chunk_id AS match,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE chunk_id = '{example_chunk}'),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT',
    options => '{{"fraction_lists_to_search": 0.25}}'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,chunk_id,match,distance
0,fannie_part_0_c17,fannie_part_0_c17,-0.999799
1,fannie_part_0_c17,fannie_part_0_c15,-0.897096
2,fannie_part_0_c17,fannie_part_0_c20,-0.889913
3,fannie_part_0_c17,fannie_part_0_c11,-0.883223
4,fannie_part_0_c17,fannie_part_0_c22,-0.877935


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [78]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Matches For An Input Embedding

In [79]:
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT',
    options => '{{"fraction_lists_to_search": 0.25}}'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,freddie_part_4_c509,freddie,-0.680526
2,Does a lender have to perform servicing functi...,freddie_part_4_c510,freddie,-0.675330
3,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
4,Does a lender have to perform servicing functi...,fannie_part_0_c326,fannie,-0.668350


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [80]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Pre-Filter Matches For An Input Embedding

In [81]:
example_gse = 'fannie'
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE gse = '{example_gse}'),
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT',
    options => '{{"fraction_lists_to_search": 0.25}}'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
2,Does a lender have to perform servicing functi...,fannie_part_0_c326,fannie,-0.668350
3,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,-0.661434
4,Does a lender have to perform servicing functi...,fannie_part_0_c240,fannie,-0.660858


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [82]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

### Get Matches: Vector Search With Vector Index - Brute Force

Rather than using the index, find the exact nearest neighbor by searching all of the embeddings with options value `use_brute_force` set to `true`:

#### Matches For A Given `chunk_id`

In [83]:
example_chunk = 'fannie_part_0_c17'
query = f'''
SELECT
    query.chunk_id AS chunk_id,
    base.chunk_id AS match,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE chunk_id = '{example_chunk}'),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT',
    options => '{{"use_brute_force":true}}'
)
ORDER BY distance
'''
ob = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
2,Does a lender have to perform servicing functi...,fannie_part_0_c326,fannie,-0.668350
3,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,-0.661434
4,Does a lender have to perform servicing functi...,fannie_part_0_c240,fannie,-0.660858


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [84]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Matches For An Input Embedding

In [85]:
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT',
    options => '{{"use_brute_force":true}}'
)
ORDER BY distance
'''
ob = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
2,Does a lender have to perform servicing functi...,fannie_part_0_c326,fannie,-0.668350
3,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,-0.661434
4,Does a lender have to perform servicing functi...,fannie_part_0_c240,fannie,-0.660858


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [86]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

#### Pre-Filter Matches For An Input Embedding

In [87]:
example_gse = 'fannie'
query = f'''
SELECT
    query.question AS question,
    base.chunk_id AS match,
    base.gse AS gse,
    distance
FROM VECTOR_SEARCH(
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE gse = '{example_gse}'),
    'embedding',
    (SELECT {question_embedding} AS embedding, '{question}' AS question),
    'embedding',
    top_k => 5,
    distance_type => 'DOT_PRODUCT',
    options => '{{"use_brute_force":true}}'
)
ORDER BY distance
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
job.to_dataframe()

,question,match,gse,distance
0,Does a lender have to perform servicing functi...,fannie_part_0_c352,fannie,-0.709984
1,Does a lender have to perform servicing functi...,fannie_part_0_c353,fannie,-0.672371
2,Does a lender have to perform servicing functi...,fannie_part_0_c326,fannie,-0.668350
3,Does a lender have to perform servicing functi...,fannie_part_0_c92,fannie,-0.661434
4,Does a lender have to perform servicing functi...,fannie_part_0_c240,fannie,-0.660858


Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [88]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'UNUSED',
 'indexUnusedReasons': [{'code': 'INDEX_SUPPRESSED_BY_FUNCTION_OPTION',
   'message': 'The vector index `row_index` of the base table `statmike-mlops-349915:applied_genai.retrieval-bigquery` was not used because use_brute_force option has been specified.',
   'baseTable': {'projectId': 'statmike-mlops-349915',
    'datasetId': 'applied_genai',
    'tableId': 'retrieval-bigquery'},
   'indexName': 'row_index'}]}

### Get Matches: Batch Matching Is The BigQuery Advantage!

Getting a match for a single input, or few inputs, is common in online applications.  Many applicationn can benefit from batch matching and BigQuery easily extends to this.  Here is an example of finding the top matching chunks from the `gse = freddie` chunks for each `gse = fannie` chunk.  Thats thousands of simoutaneous matches! 

In [89]:
query = f'''
SELECT
    query.chunk_id AS fannie_chunk_id,
    ARRAY_AGG(base.chunk_id ORDER BY distance) as matches
FROM VECTOR_SEARCH(
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE gse = 'freddie'),
    'embedding',
    (SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE gse = 'fannie'),
    'embedding',
    top_k => 2,
    distance_type => 'DOT_PRODUCT'
)
GROUP BY fannie_chunk_id
ORDER BY fannie_chunk_id
'''
job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
job.result()
results = job.to_dataframe()
results.head()

,fannie_chunk_id,matches
0,fannie_part_0_c1001,"[freddie_part_2_c347, freddie_part_2_c315]"
1,fannie_part_0_c1003,"[freddie_part_2_c420, freddie_part_2_c442]"
2,fannie_part_0_c1004,"[freddie_part_2_c424, freddie_part_2_c446]"
3,fannie_part_0_c101,"[freddie_part_5_c184, freddie_part_5_c446]"
4,fannie_part_0_c1010,"[freddie_part_2_c308, freddie_part_2_c344]"


In [90]:
results.shape

(1728, 2)

Check to see if the index was used with [vector index usage information](https://cloud.google.com/bigquery/docs/vector-index#vector_index_usage)

In [91]:
job._properties['statistics']['query']['vectorSearchStatistics']

{'indexUsageMode': 'FULLY_USED'}

How many megabytes were scanned by the job:

In [92]:
job.total_bytes_processed / (1024*1024)

112.42423439025879

What was the compute time (slot time) of the job in BigQuery in seconds:

In [93]:
job.slot_millis/1000

216.085

In [94]:
(job.ended - job.started).total_seconds()

6.177

Wow!! This found the top matches for each of the 1624 Fannie Mae chunk within the thousands of Freddie Mac chunks in only a few seconds.

---
## Retrieval Augmented Generation (RAG)

Build a simple retrieval augmented generation process that enhances a query by retrieving context.  This is done here by constructing three functions for the stages:
- `retrieve` - a function that uses an embedding to search for matching context parts, pieces of texts
    - this uses the system built earlier in this workflow!
- `augment` - prepare chunks into a prompt
- `generate` - make the llm request with the augmented prompt

A final function is used to execute the workflow of rag:
- `rag` - a function that receives the query an orchestrates the workflow through `retrieve` > `augment` > `generate`

### Clients

In [95]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')
llm = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")

### Retrieve Function

In [110]:
def retrieve_bigquery(query_embedding, n_matches = 5):

    query = f'''
    SELECT
        base.chunk_id as chunk_id,
        base.content AS content,
    FROM VECTOR_SEARCH(
        TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`,
        'embedding',
        (SELECT {question_embedding} AS embedding),
        'embedding',
        top_k => {n_matches},
        distance_type => 'DOT_PRODUCT'
    )
    ORDER BY distance
    '''
    job = bq.query(query, job_config = bigquery.QueryJobConfig(use_query_cache=False))
    job.result()
    results = job.to_dataframe()
    matches = results.to_dict('records')
    
    return matches

### Augment Function

In [111]:
def augment(matches):

    prompt = ''
    for m, match in enumerate(matches):
        prompt += f"Context {m+1}:\n{match['content']}\n\n"
    prompt += f'Answer the following question using the provided contexts:\n'

    return prompt

### Generate Function

In [112]:
def generate(prompt):

    result = llm.generate_content(prompt)

    return result

### RAG Function

In [113]:
def rag(query):
    
    query_embedding = embedder.get_embeddings([query])[0].values
    matches = retrieve_bigquery(query_embedding)
    prompt = augment(matches) + query
    result = generate(prompt)
    
    return result.text

### Example In Use

In [116]:
question

'Does a lender have to perform servicing functions directly?'

In [115]:
print(rag(question))

No, a lender does not have to perform servicing functions directly.  Context 1 explicitly states that a lender may use other organizations ("subservicers") to perform some or all of its servicing functions.  The contexts also detail the requirements and regulations surrounding these "subservicing" arrangements, including the roles of "master servicers" and Fannie Mae's approval process.



---
### Profiling Performance

Profile the timing of each step in the RAG function for sequential calls. The environment choosen for this workflow is a minimal testing enviornment so load testing (simoultaneous requests) would not be helpful.

In [117]:
profile = []

In [118]:
def rag(query, profile = profile):
    
    timings = {}
    start_time = time.time()
    
    
    # 1. Get embeddings
    embedding_start = time.time()
    query_embedding = embedder.get_embeddings([query])[0].values
    timings['embedding'] = time.time() - embedding_start

    # 2. Retrieve from Bigtable
    retrieval_start = time.time()
    matches = retrieve_bigquery(query_embedding)
    timings['retrieve_bigquery'] = time.time() - retrieval_start

    # 3. Augment the prompt
    augment_start = time.time()
    prompt = augment(matches) + query
    timings['augment'] = time.time() - augment_start

    # 4. Generate text
    generate_start = time.time()
    result = generate(prompt)
    timings['generate'] = time.time() - generate_start

    total_time = time.time() - start_time
    timings['total'] = total_time
    
    profile.append(timings)
    
    return result.text

In [119]:
print(rag(question))

No, a lender does not have to perform servicing functions directly.  A lender may use a subservicer to perform some or all of its servicing functions (Context 1).  However, there are specific requirements and approvals needed if this is done, including Fannie Mae approval of both the master servicer and the subservicer (Context 2).  The lender can also assign servicing to another lender, either concurrently with the loan sale to Fannie Mae (Context 4) or after delivery (Context 3), but this must comply with Fannie Mae's requirements.



In [120]:
profile

[{'embedding': 0.13866305351257324,
  'retrieve_bigquery': 5.720819473266602,
  'augment': 5.1975250244140625e-05,
  'generate': 1.0056006908416748,
  'total': 6.865142822265625}]

In [121]:
for i in range(100):
    response = rag(question)

### Report From Profile

In [122]:
all_timings = {}
for timings in profile:
    for key, value in timings.items():
        if key not in all_timings:
            all_timings[key] = []
        all_timings[key].append(value)

In [123]:
for key, values in all_timings.items():
    arr = np.array(values)
    print(f"Statistics for '{key}':")
    print(f"  Min: {np.min(arr):.4f} seconds")
    print(f"  Max: {np.max(arr):.4f} seconds")
    print(f"  Mean: {np.mean(arr):.4f} seconds")
    print(f"  Median: {np.median(arr):.4f} seconds")
    print(f"  Std Dev: {np.std(arr):.4f} seconds")
    print(f"  P95: {np.percentile(arr, 95):.4f} seconds")
    print(f"  P99: {np.percentile(arr, 99):.4f} seconds")
    print("")

Statistics for 'embedding':
  Min: 0.0644 seconds
  Max: 0.3224 seconds
  Mean: 0.0834 seconds
  Median: 0.0787 seconds
  Std Dev: 0.0274 seconds
  P95: 0.1051 seconds
  P99: 0.1573 seconds

Statistics for 'retrieve_bigquery':
  Min: 3.8252 seconds
  Max: 8.2738 seconds
  Mean: 4.6543 seconds
  Median: 4.6210 seconds
  Std Dev: 0.5096 seconds
  P95: 5.2126 seconds
  P99: 5.8101 seconds

Statistics for 'augment':
  Min: 0.0000 seconds
  Max: 0.0001 seconds
  Mean: 0.0000 seconds
  Median: 0.0000 seconds
  Std Dev: 0.0000 seconds
  P95: 0.0000 seconds
  P99: 0.0001 seconds

Statistics for 'generate':
  Min: 0.6410 seconds
  Max: 3.2811 seconds
  Mean: 0.9653 seconds
  Median: 0.9153 seconds
  Std Dev: 0.2708 seconds
  P95: 1.2045 seconds
  P99: 1.4922 seconds

Statistics for 'total':
  Min: 4.7843 seconds
  Max: 9.1097 seconds
  Mean: 5.7030 seconds
  Median: 5.6160 seconds
  Std Dev: 0.5614 seconds
  P95: 6.3193 seconds
  P99: 7.9213 seconds



---
## Low Latency Vector Search With BigQuery

Did you know that Vertex AI Feature Store synchronizes BigQuery sources to an online store with a high speed client?  And that embeddings matching is a built-in feature of the online store?  Continue on to the next workflow in the [Retrieval](./readme.md) section that extends this workflow to Vertex AI Feature Store: [Retrieval - Vertex AI Feature Store](Retrieval%20-%20Vertex%20AI%20Feature%20Store.ipynb).

---
## Remove Resources

The resources created above in BigQuery will persist unless deleted.  To remove the table created above uncomment and run the following cell:

In [38]:
#bq.delete_table(bq_table)